# Stock Market Prediction

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error as mae
from matplotlib import style

#### Initial overlook of data and transforming Date column

In [2]:
stock = pd.read_csv('sphist.csv')
stock2 = stock.copy()

In [3]:
stock.head()

,Date,Open,High,Low,Close,Volume,Adj Close
0,2015-12-07,2090.419922,2090.419922,2066.780029,2077.070068,4043820000,2077.070068
1,2015-12-04,2051.239990,2093.840088,2051.239990,2091.689941,4214910000,2091.689941
2,2015-12-03,2080.709961,2085.000000,2042.349976,2049.620117,4306490000,2049.620117
3,2015-12-02,2101.709961,2104.270020,2077.110107,2079.510010,3950640000,2079.510010
4,2015-12-01,2082.929932,2103.370117,2082.929932,2102.629883,3712120000,2102.629883


In [4]:
stock.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16590 entries, 0 to 16589
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       16590 non-null  object 
 1   Open       16590 non-null  float64
 2   High       16590 non-null  float64
 3   Low        16590 non-null  float64
 4   Close      16590 non-null  float64
 5   Volume     16590 non-null  int64  
 6   Adj Close  16590 non-null  float64
dtypes: float64(5), int64(1), object(1)
memory usage: 907.4+ KB


In [5]:
## Turn date into datetime object and sort dataframe by date ascending
stock['Date'] = pd.to_datetime(stock['Date'])
stock = stock.sort_values(by='Date',ascending=True, ignore_index=True)

stock2['Date'] = pd.to_datetime(stock2['Date'])
stock2 = stock2.sort_values(by='Date',ascending=True, ignore_index=True)

stock.head(3)

,Date,Open,High,Low,Close,Volume,Adj Close
0,1950-01-03,16.66,16.66,16.66,16.66,1260000,16.66
1,1950-01-04,16.85,16.85,16.85,16.85,1890000,16.85
2,1950-01-05,16.93,16.93,16.93,16.93,2550000,16.93


In [6]:
stock.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16590 entries, 0 to 16589
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       16590 non-null  datetime64[ns]
 1   Open       16590 non-null  float64       
 2   High       16590 non-null  float64       
 3   Low        16590 non-null  float64       
 4   Close      16590 non-null  float64       
 5   Volume     16590 non-null  int64         
 6   Adj Close  16590 non-null  float64       
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 907.4 KB


## Generating Indicators
We'll choose four indicators which can tell us the stock price for a current day (or close to). The four indicators we will choose are:

- The average price from the past 5 days
- The average price from the past 30 days
- The standard deviation from the past 5 days
- The standard deviation from the past 30 days

In [7]:
stock['avg_5'] = 0
stock['avg_30'] = 0
stock['std_5'] = 0
stock['std_30'] = 0

#Iterate through the rows of dataframe
for idx, row in stock.iterrows():
    # prev_5 is a dataframe equal to the current index-5 rows, same with prev_30
    prev_5 = stock.iloc[idx-5:idx,:]
    prev_30 = stock.iloc[idx-30:idx,:]
    
    # calculate mean and std for each of the 4 indicators
    stock.loc[idx,'avg_5'] = prev_5['Close'].mean()
    stock.loc[idx,'avg_30'] = prev_30['Close'].mean()
    
    stock.loc[idx,'std_5'] = prev_5['Close'].std()
    stock.loc[idx,'std_30'] = prev_30['Close'].std()

In [8]:
stock.head(31)

,Date,Open,High,Low,Close,Volume,Adj Close,avg_5,avg_30,std_5,std_30
0,1950-01-03,16.660000,16.660000,16.660000,16.660000,1260000,16.660000,NaN,NaN,NaN,NaN
1,1950-01-04,16.850000,16.850000,16.850000,16.850000,1890000,16.850000,NaN,NaN,NaN,NaN
2,1950-01-05,16.930000,16.930000,16.930000,16.930000,2550000,16.930000,NaN,NaN,NaN,NaN
3,1950-01-06,16.980000,16.980000,16.980000,16.980000,2010000,16.980000,NaN,NaN,NaN,NaN
4,1950-01-09,17.080000,17.080000,17.080000,17.080000,2520000,17.080000,NaN,NaN,NaN,NaN
5,1950-01-10,17.030001,17.030001,17.030001,17.030001,2160000,17.030001,16.900,NaN,0.157956,NaN
6,1950-01-11,17.090000,17.090000,17.090000,17.090000,2630000,17.090000,16.974,NaN,0.089051,NaN
7,1950-01-12,16.760000,16.760000,16.760000,16.760000,2970000,16.760000,17.022,NaN,0.067602,NaN
8,1950-01-13,16.670000,16.670000,16.670000,16.670000,3330000,16.670000,16.988,NaN,0.134796,NaN
9,1950-01-16,16.719999,16.719999,16.719999,16.719999,1460000,16.719999,16.926,NaN,0.196545,NaN


## Splitting Up the Data
Since we're computing using historical data, we should get rid of any columns with NaN values. This would be any date before February 15, 1950.

In [9]:
stock.isnull().sum()

Date          0
Open          0
High          0
Low           0
Close         0
Volume        0
Adj Close     0
avg_5         5
avg_30       30
std_5         5
std_30       30
dtype: int64

We can see that there are no missing values except the first 30.

#### Drop rows with many NaN values

In [10]:
stock = stock[stock['Date'] > datetime(year=1950, month=2, day=14)]

In [11]:
stock.head(3)

,Date,Open,High,Low,Close,Volume,Adj Close,avg_5,avg_30,std_5,std_30
30,1950-02-15,17.059999,17.059999,17.059999,17.059999,1730000,17.059999,17.204,16.976667,0.084440,0.197978
31,1950-02-16,16.990000,16.990000,16.990000,16.990000,1920000,16.990000,17.170,16.990000,0.103442,0.189191
32,1950-02-17,17.150000,17.150000,17.150000,17.150000,1940000,17.150000,17.126,16.994667,0.126413,0.187336


#### Generate Train and Test Dataframes
Since we are predicting for the future, our train set will include earlier entries of the data while the test set will contain the latter end of the dataset.

In [12]:
train = stock[stock['Date'] < datetime(year=2013, month=1, day=1)]
test = stock[stock['Date'] >= datetime(year=2013, month=1, day=1)]

In [13]:
train.tail(2)

,Date,Open,High,Low,Close,Volume,Adj Close,avg_5,avg_30,std_5,std_30
15849,2012-12-28,1418.099976,1418.099976,1401.579956,1402.430054,2426680000,1402.430054,1427.685986,1410.265332,10.208568,23.215373
15850,2012-12-31,1402.430054,1426.739990,1398.109985,1426.189941,3204330000,1426.189941,1419.434009,1411.830001,10.701861,20.858522


In [14]:
test.head(2)

,Date,Open,High,Low,Close,Volume,Adj Close,avg_5,avg_30,std_5,std_30
15851,2013-01-02,1426.189941,1462.430054,1426.189941,1462.420044,4202600000,1462.420044,1418.641992,1414.258667,9.820801,17.834740
15852,2013-01-03,1462.420044,1465.469971,1455.530029,1459.369995,3829730000,1459.369995,1425.793994,1417.676668,22.261321,16.852563


Jan 1, 2013 is not a valid entry in this dataset, this is most likely because the stock market was closed that day.

## Making Predictions
Let's now train a model using the train set, and test it on the test set

In [15]:
lr = LinearRegression()
features = ['avg_5','avg_30','std_5','std_30']

lr.fit(train[features],train['Close'])

prediction = lr.predict(test[features])

mean_abs_error = mae(test['Close'],prediction)

test = test.reset_index()

test['Predicted closing price'] = prediction

In [16]:
test['Error'] =  round(abs(test["Close"] - test["Predicted closing price"]),3)
test['Error %'] =  round(abs(test["Error"] / test["Close"])*100,3)

In [17]:
test

,index,Date,Open,High,Low,Close,Volume,Adj Close,avg_5,avg_30,std_5,std_30,Predicted closing price,Error,Error %
0,15851,2013-01-02,1426.189941,1462.430054,1426.189941,1462.420044,4202600000,1462.420044,1418.641992,1414.258667,9.820801,17.834740,1418.746604,43.673,2.986
1,15852,2013-01-03,1462.420044,1465.469971,1455.530029,1459.369995,3829730000,1459.369995,1425.793994,1417.676668,22.261321,16.852563,1424.042661,35.327,2.421
2,15853,2013-01-04,1459.369995,1467.939941,1458.989990,1466.469971,3424290000,1466.469971,1433.702002,1420.092668,26.274326,17.470824,1431.751457,34.719,2.368
3,15854,2013-01-07,1466.469971,1466.469971,1456.619995,1461.890015,3304970000,1461.890015,1443.376001,1422.714665,27.945242,18.339803,1441.747344,20.143,1.378
4,15855,2013-01-08,1461.890015,1461.890015,1451.640015,1457.150024,3601600000,1457.150024,1455.267993,1425.076664,16.453319,18.678333,1456.251838,0.898,0.062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
734,16585,2015-12-01,2082.929932,2103.370117,2082.929932,2102.629883,3712120000,2102.629883,2087.024023,2073.984998,3.916109,24.654181,2089.172773,13.457,0.640
735,16586,2015-12-02,2101.709961,2104.270020,2077.110107,2079.510010,3950640000,2079.510010,2090.231982,2076.283993,7.956808,23.970453,2091.721972,12.212,0.587
736,16587,2015-12-03,2080.709961,2085.000000,2042.349976,2049.620117,4306490000,2049.620117,2088.306006,2077.908659,9.333599,22.378095,2089.163999,39.544,1.929
737,16588,2015-12-04,2051.239990,2093.840088,2051.239990,2091.689941,4214910000,2091.689941,2080.456006,2078.931331,19.599946,20.183769,2078.779967,12.910,0.617


In [18]:
mean_abs_error

16.218518244098387

## Adding extra indicators
Let's add two more indicators and see how the mean absolute error changes

In [19]:
stock2['avg_5'] = 0
stock2['avg_30'] = 0
stock2['avg_365'] = 0
stock2['std_5'] = 0
stock2['std_30'] = 0
stock2['std_365'] = 0

#Iterate through the rows of dataframe
for idx, row in stock2.iterrows():
    # prev_5 is a dataframe equal to the current index-5 rows, same with prev_30
    prev_5 = stock2.iloc[idx-5:idx,:]
    prev_30 = stock2.iloc[idx-30:idx,:]
    prev_365 = stock2.iloc[idx-365:idx,:]

    # calculate mean and std for each of the 6 indicators
    stock2.loc[idx,'avg_5'] = prev_5['Close'].mean()
    stock2.loc[idx,'avg_30'] = prev_30['Close'].mean()
    stock2.loc[idx,'avg_365'] = prev_365['Close'].mean()

    stock2.loc[idx,'std_5'] = prev_5['Close'].std()
    stock2.loc[idx,'std_30'] = prev_30['Close'].std()
    stock2.loc[idx,'std_365'] = prev_365['Close'].std()

In [21]:
stock2 = stock2[stock2['Date'] > datetime(year=1951, month=6, day=18)]

In [22]:
train2 = stock2[stock2['Date'] < datetime(year=2013, month=1, day=1)]
test2 = stock2[stock2['Date'] >= datetime(year=2013, month=1, day=1)]

In [23]:
lr = LinearRegression()
features2 = ['avg_5','avg_30','std_5','std_30','avg_365','std_365']

lr.fit(train2[features2],train2['Close'])

prediction = lr.predict(test2[features2])

mean_abs_error = mae(test2['Close'],prediction)

test2 = test2.reset_index()

test2['Predicted closing price'] = prediction

In [24]:
test2['Error'] =  round(abs(test2["Close"] - test2["Predicted closing price"]),3)
test2['Error %'] =  round(abs(test2["Error"] / test2["Close"])*100,3)

In [25]:
test2

,index,Date,Open,High,Low,Close,Volume,Adj Close,avg_5,avg_30,avg_365,std_5,std_30,std_365,Predicted closing price,Error,Error %
0,15851,2013-01-02,1426.189941,1462.430054,1426.189941,1462.420044,4202600000,1462.420044,1418.641992,1414.258667,1327.534055,9.820801,17.834740,90.463948,1418.715248,43.705,2.989
1,15852,2013-01-03,1462.420044,1465.469971,1455.530029,1459.369995,3829730000,1459.369995,1425.793994,1417.676668,1327.908247,22.261321,16.852563,90.738976,1423.951089,35.419,2.427
2,15853,2013-01-04,1459.369995,1467.939941,1458.989990,1466.469971,3424290000,1466.469971,1433.702002,1420.092668,1328.224877,26.274326,17.470824,90.995857,1431.631975,34.838,2.376
3,15854,2013-01-07,1466.469971,1466.469971,1456.619995,1461.890015,3304970000,1461.890015,1443.376001,1422.714665,1328.557617,27.945242,18.339803,91.279049,1441.605833,20.284,1.388
4,15855,2013-01-08,1461.890015,1461.890015,1451.640015,1457.150024,3601600000,1457.150024,1455.267993,1425.076664,1328.898603,16.453319,18.678333,91.544368,1456.133567,1.016,0.070
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
734,16585,2015-12-01,2082.929932,2103.370117,2082.929932,2102.629883,3712120000,2102.629883,2087.024023,2073.984998,2035.531178,3.916109,24.654181,64.370261,2087.999571,14.630,0.696
735,16586,2015-12-02,2101.709961,2104.270020,2077.110107,2079.510010,3950640000,2079.510010,2090.231982,2076.283993,2035.914082,7.956808,23.970453,64.352527,2090.530898,11.021,0.530
736,16587,2015-12-03,2080.709961,2085.000000,2042.349976,2049.620117,4306490000,2049.620117,2088.306006,2077.908659,2036.234356,9.333599,22.378095,64.277554,2087.978896,38.359,1.872
737,16588,2015-12-04,2051.239990,2093.840088,2051.239990,2091.689941,4214910000,2091.689941,2080.456006,2078.931331,2036.507343,19.599946,20.183769,64.121622,2077.578077,14.112,0.675


In [26]:
mean_abs_error

16.214387095799893

## Conclusion
As we can see, adding two extra indicators bought the MAE down by a very slight amount. We can assume that is because the market can change a lot throughout 365 days which may not bring the best indication to the current days closing price.